<a href="https://colab.research.google.com/github/sheelaj123/Prompt_Engineering_and_LLMs_with_LangChain/blob/main/Guard_railing_adversarial_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color =red>Adversarial prompts

Adversarial prompting is an important topic in prompt engineering as it could help to understand the risks and safety issues involved with LLMs.
While building LLMs, it's important to protect against prompt attacks that could bypass safety guardrails and break the guiding principles of the model.
Following are some of the adversarial prompting techniques:

1. >Prompt Injection
2. >Prompt leaking
3. >Jailbreaking


#<font color =red>What is prompt Hacking .?

##it is type of attack that exploits the vulnerabilities of LLM  by manipulating their input or prompts.


###Traditional hacking --> exploit software vulnerabilities

                   BUT
###Prompt hacking      --> crafting prompts to deceive the LLM




##<font color =red>Let's understands with examples--

###Guard railing the adversarial Prompts

####The code below demonstrates how to add Guardrails to add a layer of security around LangChain components.

#<font color =red>Step: 1 Installing some imp packeges-

In [1]:
!pip install guardrails-ai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00


In [3]:
!pip install huggingface_hub

#<font color =red>Step -2 Creating a RAIL spec

In [4]:
rail_spec = """
<rail version="0.1">
<output>
   <object name="customer_info">
       <string name="gender" description="Cusotmer's gender" />
       <integer name="age" format="valid-range: 0 100" />
       <string name="Issue" description="Issues that the Customer is currently experiencing" />
   </object>
</output>
<prompt>
Given the following Advisor's notes about the customer, please extract a dictionary that contains the Customer's information.
{{Advisor_notes}}
@complete_json_suffix_v2
</prompt>
</rail>
"""


In [17]:
from rich import print
from getpass import getpass
import os
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts import PromptTemplate
#set the Huggingface API token
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
#Connect to LLM
repo_id1 = "microsoft/phi-2"


··········


#Step -3

#<font color =red>⚛Create a GuardrailsOutputParser ⚛

contains a Guard object, which can be used to access the prompt and output schema. E.g., here is the compiled prompt that is stored in GuardrailsOutputParser:

In [18]:
llm = HuggingFaceHub(repo_id=repo_id1, model_kwargs={"temperature":0.7, "max_length":200})
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=llm)

In [19]:
print(output_parser.guard.base_prompt)

Given the following Advisor's notes about the customer, please extract a dictionary that contains the Customer's 
information.
{{Advisor_notes}}
@complete_json_suffix_v2

#<font color =red>Step -4 Create a Prompt template ⏭

Query the LLM and get formatted, validated and corrected output

In [ ]:
prompt = PromptTemplate(template=output_parser.guard.base_prompt,input_variables=output_parser.guard.prompt.variable_names)
Chain = LLMChain(prompt=prompt, llm=llm)
Advisor_notes= """49 y/o Male with his Annual income of 10K USD has invested on real estate worth 20K USD"""
output = llm(prompt.format_prompt(Advisor_notes=Advisor_notes).to_string())
print(output_parser.parse(output))


#<font color =red>Topic over now, thanks for visiting... <> Happy learning